# Python Geospatial Libraries

An introduction to some of the core python libraries that provide geospatial capabilities. The following libraries are not a full list of geospatial python libraries, not even close, but a selection to provide users with an idea of what can be acomplished with python.  

In [ ]:
# click the play button to the left to run this code
print("Lets get started!")

## Shapely
___

Shapely is a Python library for manipulation and analysis of planar geometric objects, which are based on the widely adopted GEOS library (the engine of PostGIS).

Shapely is particularly good for tasks that involve geometries like points, lines, and polygons. It supports operations like buffer, union, intersection, and difference, among others.

```
> pip install shapely
```

#### Basic Geometric Operations

In [ ]:
from shapely.geometry import Point, LineString, Polygon

# Create a point
point = Point(0, 0)
# the positions are (x, y)

# Create a line
line = LineString([(0, 0), (1, 1), (2, 0)])

# Create a polygon
polygon = Polygon([(0, 0), (1, 1), (1, 0)])

Lets now look at the objects created, switch the "point" below with the polygon variable.

In [ ]:
line

#### Buffer

With these objects we can use the pre-build functions to perform different operations.

In [ ]:
# Create a buffer around the point
buffer = point.buffer(1.0)

print(buffer.area)

In [ ]:
buffer

#### Object Attributes + Methods

Shapely object have a number of built in methods (functions) and attributes that can be easily accessed. Examples are "length", "area", "bounds", "centroid".



> **TO DO:** in the same way we found the area of the buffer above, find the boundary of the buffer.


In [ ]:
print("buffer boundary: \n")

# type below to access the boundary

#### Union

In [ ]:
# Create another point
point2 = Point(2, 2)

# Create union of the two points (result will be a MultiPoint geometry)
union_result = point.union(point2)

In [ ]:
union_result

Intersection

In [ ]:
# Create another line
line2 = LineString([(0, 0), (0, 2)])

# Find intersection (result will be a Point or LineString depending on the overlap)
intersection_result = line.intersection(line2)

In [ ]:
line # 0,0 1,1 2,0

In [ ]:
line2 # 0,0 0,2

In [ ]:
# can access the xy of the point where these two lines cross
print(intersection_result.coords.xy)

#### Relationships Between Objects

Shapely can also test for different types of spatial relationships between geometries.

In [ ]:
# Check if a point is within a polygon
print(point.within(polygon))

# Check if a polygon contains a point
print(polygon.contains(point))

This is just a brief introduction, and Shapely has many more capabilities to explore, such as geometry validation, simplification, and precise configuration of operations. It's a powerful library for anyone working with geospatial data in Python.

## Fiona & PyProj
___
**Fiona** is a library for reading and writing vector data (i.e., data that is made up of points, lines, and polygons).

It provides a straightforward and Pythonic interface to work with geospatial data built on top of the powerful GDAL.

Can handle multiple formats like Shapefiles, GeoJSON, and others.

```
> pip install fiona
```

**PyProj** is a Python interface to the proj library, which is used for cartographic projections and coordinate transformations.

PyProj is often used in combination with other geospatial libraries like Fiona and Shapely.

```
> pip install pyproj
```

#### Reading Data

The feature object is a Python dictionary complying with the GeoJSON format.

We don't have a directory to read from but just so you can see how little code this is I wanted to add it.

In [ ]:
import fiona

# Opening  the shapefile would look like this
"""
with fiona.open("path/to/file/coutwildrnp.shp", "r") as src:
    # Loop through the features and print them
    for feature in src:
        print(feature)
"""

#### Writing Data



In [ ]:
# Schema of the new shapefile
schema = {
    'geometry': 'Point',
    'properties': {'id': 'int', 'name': 'str'}
}

# Data for the new shapefile
features = [
    {
        'geometry': {'type': 'Point', 'coordinates': (0, 0)},
        'properties': {'id': 1, 'name': 'First Point'}
    },
    {
        'geometry': {'type': 'Point', 'coordinates': (1, 1)},
        'properties': {'id': 2, 'name': 'Second Point'}
    }
]

# Write the shapefile would look like the code below
"""
with fiona.open('file/path/output.shp', 'w', driver='ESRI Shapefile', schema=schema) as dst:
    for feature in features:
        dst.write(feature)
"""

#### Basic Coordinate Transformations - PyProj

PyProj allows you to convert coordinates from one CRS (Coordinate Reference System) to another. Here's an example to convert coordinates from WGS84 (EPSG:4326) to Web Mercator (EPSG:3857):

In [ ]:
from pyproj import Transformer

# Create a transformer object
transformer = Transformer.from_crs("EPSG:4326", "EPSG:26917")

# Perform the transformation
x, y = transformer.transform(50, -80)  # Note that the order is (lat, lon)

# Note that this output is radians
print(f"Transformed coordinates: x={x}, y={y}")

#### Geodetic Calculations

PyProj can also be used to perform various geodetic calculations like distance, forward, and inverse geodetic problems.

In [ ]:
from pyproj import Geod

# Create a Geod object
geod = Geod(ellps="WGS84")

# Calculate distance and angles between two points
lat1, lon1 = 37.75, -122.45  # San Francisco
lat2, lon2 = 34.05, -118.25  # Los Angeles

az12, az21, distance = geod.inv(lon1, lat1, lon2, lat2)

Printing the calcualted metrics:

In [ ]:
print(f"Distance: {distance} meters")
print(f"Forward azimuth: {az12} degrees")
print(f"Backward azimuth: {az21} degrees")

## GeoPandas
____

Makes working with geospatial data in python easier by extending datatypes used by pandas to allow spatial operations on geometric types. GeoPandas enables you to easily do operations in python, such as joins or cuts, that would otherwise require a spatial database such as PostGIS.

- Geometric operations are performed by shapely.
- Geopandas further depends on fiona for file access and matplotlib for plotting.

```
> pip install geopandas
```

#### Reading Spatial Data from a File

GeoPandas can read various types of spatial data files, such as shapefiles, GeoJSON, and more. Reading a shapefile is as simple as the code below.

In [ ]:
import geopandas as gpd
import requests
from io import StringIO

# URL for a GeoJSON file
url = "https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json"

# Fetch the content
response = requests.get(url)
data = response.text

# Read the GeoJSON content into a GeoDataFrame
gdf = gpd.read_file(StringIO(data), driver="GeoJSON")

# Print the first 5 rows
print(gdf.head())

#### Basic Operations

**Filtering by Geometry** is how you could filter all polygons that overlap with your target polygon, this can also be done with points within a certain boundary.

First step is to create a new polygon.

In [ ]:
from shapely.geometry import shape
import json

# Sample GeoJSON string for a polygon
geojson_str = """
{
    "type": "Polygon",
    "coordinates": [
        [
            [-79.45015642620336,43.64738992644382],
            [-79.3698344704733,43.64738992644382],
            [-79.3698344704733,43.683807516070885],
            [-79.45015642620336,43.683807516070885],
            [-79.45015642620336,43.64738992644382]
        ]
    ]
}
"""

# Load the GeoJSON string and parse it into a geometry
geometry = shape(json.loads(geojson_str))

# Now you can create a GeoDataFrame
polygon_gdf = gpd.GeoDataFrame({'geometry': [geometry]})

print(polygon_gdf)

Next we can find the overlapping polygons.

In [ ]:
# Use spatial join to find polygons in gdf that overlap with single_polygon_gdf
overlapping = gpd.sjoin(gdf, polygon_gdf, how="inner", op="intersects")

# I left the error below in to show how smart geopandas is

overlapping.head()

Visualization

GeoPandas also makes it simple to plot your data. To do this we're also going to take advantage of the **matplotlib** python library.

In [ ]:
import matplotlib.pyplot as plt

# Plot all country polygons
gdf.plot(color='red')
plt.show()

# Plot our custom polygon
polygon_gdf.plot(color='blue')
plt.show()

# Plot the country polygon overlapping with our custom polygon
overlapping.plot(color='green')
plt.show()

## Folium

Used to visualize data on an interactive leaflet map.
Enables both the binding of data to a map for choropleth visualizations as well as passing rich vector/raster/HTML visualizations as markers on the map.

```
> pip install folium
```

#### Creating a Basic Interactive Map

In [ ]:
import folium

# Create a base map
fol_map_1 = folium.Map(location=[43.72, -79.38], tiles='OpenStreetMap', zoom_start=10)

# Display the map
fol_map_1

Switch the "tiles" option to one of the following:

- stamenterrain
- cartodbpositron
- cartodbdark_matter
- stamenwatercolor
- stamentoner

#### Adding context to polygons with base maps

Lets use the existing polygon in geopandas to get a better idea of where it is.

In [ ]:
# Add GeoDataFrame to the map
folium.GeoJson(geometry).add_to(fol_map_1)

fol_map_1

#### Mapping some data

First lets make a new set of polygons that have an additional value

In [ ]:
# Define polygons for each region
vancouver_poly = Polygon([[-123.3, 48.8], [-122.2, 48.8], [-122.2, 49.7], [-123.3, 49.7]])
calgary_poly = Polygon([[-114.4, 50.7], [-113.5, 50.7], [-113.5, 51.3], [-114.4, 51.3]])
toronto_poly = Polygon([[-80.0, 43.3], [-78.5, 43.3], [-78.5, 44.1], [-80.0, 44.1]])

# Create a GeoDataFrame
data = {
    'region': ['Vancouver', 'Calgary', 'Toronto'],
    'Coolness': [250, 50, 500],
    'geometry': [vancouver_poly, calgary_poly, toronto_poly]
}

gdf = gpd.GeoDataFrame(data)

# Convert GeoDataFrame to GeoJSON
geojson = gdf.to_json()

print(geojson)

Now lets create a new map and add our data to it.

In [ ]:
# Create the base map
fol_map_2 = folium.Map(location=[55, -106], tiles="cartodbdark_matter", zoom_start=4)

# Add the Choropleth layer to the map
folium.Choropleth(
    geo_data=gdf.to_json(),
    data=gdf,
    columns=['region', 'Coolness'],
    key_on='feature.properties.region',
    fill_color='YlGn',
    fill_opacity=0.5,
    line_opacity=0,
    legend_name='Coolness in Regions'
).add_to(fol_map_2)

fol_map_2

# Basic Analysis with Leafmap

In [ ]:
!pip install leafmap

We can also create maps that have dynamic elements, like the polygon fill colour.

In [ ]:
import random
import leafmap

m_2 = leafmap.Map(center=[0, 0], zoom=2)

url = "https://raw.githubusercontent.com/opengeos/leafmap/master/examples/data/countries.geojson"

# a function to select random colors
def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange', "blue"]),
    }


m_2.add_geojson(url, layer_name="Countries", style_callback=random_color)

# display the map
m_2

Now we can create an interactive map of niagara beaches! There is also built in support for whitebox tools.

The information is drawn directly from a file hosted on an open data portal:
https://niagaraopendata.ca/dataset/beaches

In [ ]:


# collect the data from the web - its a geojson
geojson_url = "https://niagaraopendata.ca/dataset/1ca52a3b-03a9-412f-a10a-59ddab0743c5/resource/15db2b16-abe9-469f-9441-80fc3e355b8d/download/beaches.json"

# create the map
m = leafmap.Map(center=[43.052, -79.441], zoom=9)

# add the point layer
m.add_point_layer(geojson_url, popup=["BEACH_NAME", "BEACH_TYPE", "BEACH_PARKING", "BEACH_WASHROOM"], layer_name="Beaches")

# display the map
m

## PySAL Examples
---

PySAL provides a very helpful examples package that makes exploring the different libraries very easy.

In [ ]:
!pip install libpysal

In [ ]:
import libpysal.examples

There are several open data sources available to us that we can use in analysis. We can review them in a dataframe.

In [ ]:
libpysal.examples.summary()

In [ ]:
df = libpysal.examples.available()

df.head(15)

We can look at the details of one dataset were interested in.

In [ ]:
libpysal.examples.explain("AirBnB")

Its very easy to download the data or access the direct link to its source.

In [ ]:
airbnb = libpysal.examples.load_example("AirBnB")

In [ ]:
airbnb_url = libpysal.examples.get_url('AirBnB')

airbnb_url

View all the files available to us within the zip folder.

In [ ]:
airbnb.get_file_list()

Let's load one of the files and put it on a chart.

In [ ]:
airbnb_df = gpd.read_file(airbnb.get_path('airbnb_Chicago 2015.shp'))

%matplotlib inline
airbnb_df.plot()

Let's explore the dataframe to make sure we have all the information were interested in.

In [ ]:
airbnb_df.head()

#### Is there a relationship between the income Price Per Person of an AirBnB and the Hardship Index?


In [ ]:
import matplotlib.pyplot as plt

# setup the chart with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))

# plot the price per person
airbnb_df.plot(column='price_pp', cmap='viridis', legend=True, ax=ax1)
ax1.set_title('Price Per Person')

# plot the poverty
airbnb_df.plot(column='harship_in', cmap='inferno', legend=True, ax=ax2)
ax2.set_title('Hardship Index')

---

# OSMnx: Python for Street Networks
---

OSMnx is a Python package to retrieve, model, analyze, and visualize street networks from OpenStreetMap.

Users can download and model walkable, drivable, or bikeable urban networks with a single line of Python code, and then easily analyze and visualize them.

```
> pip install osmnx
```

This time we actually do need to install it, since its not pre-installed in Colab.

In [ ]:
# pip install the OSMnx package
!pip install osmnx

#### Get administrative place boundaries and shapefiles

To start we can read areas directly from open street maps.

In [ ]:
import osmnx as ox

# select the places we want
places = ox.geocode_to_gdf(['Ontario, Canada', 'Quebec, Canada', 'Manitoba, Canada'])
places = ox.project_gdf(places)

# plot the areas
ax = places.plot()
_ = ax.axis('off')

In [ ]:
city = ox.geocode_to_gdf('Toronto, Ontario, Canada')
ax = ox.project_gdf(city).plot()
_ = ax.axis('off')

#### Download and model street networks

OSMnx lets you download street network data and build topologically-corrected street networks, project and plot the networks, and save the street network as SVGs, GraphML files, or shapefiles for later use.

Streets can be selected in the following ways:

- a bounding box
- a lat-long point plus a distance
- an address plus a distance
- a polygon of the desired street network’s boundaries
- a place name or list of place names

In [ ]:
# selecting a smaller city for speed
niagara_city = ox.graph_from_place('Niagara Falls, Ontario, Canada', network_type='drive')

ox.plot_graph(niagara_city)

#### Getting building sets for cities

We can retrieve all the polygons within a city to export.

In [ ]:
# Retrieve buildings in a neighborhood
buildings = ox.features_from_place("St. Catharines, Ontario, Canada", tags={"building": True})
# this returns a dataframe we can plot

ox.project_gdf(buildings).plot(color='blue')

plt.show()

#### Analyzing the network

Let's find the shortest path between two nodes

In [ ]:
import networkx as nx

# Get the street network for a place
welland_graph = ox.graph_from_place('Welland, Ontario, Canada')

We can select individual nodes as our start and end points.

In [ ]:
# Select two nodes and compute the shortest path
orig_node = list(welland_graph.nodes())[5]
dest_node = list(welland_graph.nodes())[300]

# using the networkx package
route = nx.shortest_path(welland_graph, orig_node, dest_node, weight='length')

ox.plot_graph_route(welland_graph, route)